# Recommendation System

## Задание 1

Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). Оставь в датасете только фильмы, которые вышли в "релиз".\
Выведи количество фильмов, оставшихся после фильтрации.

In [14]:
import pandas as pd
import numpy as np

In [15]:
df1=pd.read_csv('/content/tmdb_5000_credits (1).csv')
df2=pd.read_csv('/content/tmdb_5000_movies.csv')

In [16]:
df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')

In [17]:
df2.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [18]:
df2 = df2[df2.status == 'Released']

In [19]:
df2.shape

(4795, 23)

## Задание 2

Самый наивный подход к рекомендации фильмов - рекомендовать фильмы с лучшими оценками пользователей. Фильмы, которые пользуются большей популярностью и признанием критиков, с большей вероятностью понравятся среднему зрителю.

Для справедливой оценки фильмов возьмем текущую рейтинговую систему IMDB (weighted rating (WR)), которая рассчитывается по формуле:
$$WR = \frac{v}{v + m} ⋅ R + \frac{m}{v + m} ⋅ C$$
$v$ - количество голосов \
$m$ - количество голосов для включения в финальную таблицу \
$R$ - средняя оценка \
$C$ - средняя оценка всех фильмов

Имплементируй функцию `weighted_rating`. С её помощью расcчитай рейтинг для каждого фильма и сохрани его в колонку `simple_score`.\
Выведи топ-5 фильмов по получившемуся рейтингу.
> В качестве параметра $m$ выбери 95-й квантиль количества голосов.

In [20]:
C= df2['vote_average'].mean()
C

6.093305526590198

In [21]:
m= df2['vote_count'].quantile(0.95)
m

1846.600000000004

In [22]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]
q_movies.shape

(480, 23)

In [23]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']

    return (v/(v+m) * R) + (m/(m+v) * C)

In [24]:
q_movies['simple_score'] = q_movies.apply(weighted_rating, axis=1)

In [25]:
q_movies = q_movies.sort_values('simple_score', ascending=False)


q_movies[['title', 'vote_count', 'vote_average', 'simple_score']].head(5)

,title,vote_count,vote_average,simple_score
1881,The Shawshank Redemption,8205,8.5,8.057861
662,Fight Club,9413,8.3,7.938097
65,The Dark Knight,12002,8.2,7.919089
3232,Pulp Fiction,8428,8.3,7.903402
96,Inception,13752,8.1,7.862443


## Задание 3

Такой подход к рекомендациям очень наивен, так как не учитывает информацию о самом фильме (жанр, режиссер, описание, актеры и т.п). \
**Content Based Filtering** (Фильтрация на основе содержания) - тип рекомендательной системы, которая предлагает пользователям похожие элементы на основе конкретного элемента. Общая идея этих рекомендательных систем заключается в том, что если человеку понравился определенный товар, то ему понравится и похожий на него товар.

<center><img src="../misc/images/content.png" alt= “” width="300" height="500">

Реализуем алгоритм рекомендации на основе описания фильма. Для это требуется провести предобработку текста:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Расcчитай [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильма

Выведи размер получившейся матрицы Tf-Idf

> Для [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) используйте параметры по умолчанию

In [26]:
df2['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
tfidf = TfidfVectorizer(stop_words='english')

In [29]:
df2['overview'] = df2['overview'].fillna('')

In [30]:
tfidf_matrix = tfidf.fit_transform(df2['overview'])

In [31]:
tfidf_matrix.shape

(4795, 20970)

## Задание 4

Теперь тебе необходимо вычислить показатель сходства между описаниями фильмов. Используем косинусное расстояние, оно рассчитывается по формуле:
$$cos(Θ) = \frac{A ⋅ B}{∥A∥ ∥B∥} = \frac{ Σ_{i=1}^{n} A_i ⋅ B_i } { \sqrt{Σ_{i=1}^{n}A_{i}^{2}} ⋅ {\sqrt{Σ_{i=1}^{n}B_{i}^{2}}}}$$
Но поскольку мы использовали векторизатор TF-IDF на предыдущем шаге, достаточно вычислить скалярное произведение, которое и даст оценку косинусного сходства. Рассчитать его можно через [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html). Результат сохрани в переменную `cosine_sim`.

Выведи размер получившейся матрицы `cosine_sim`.

In [32]:
from sklearn.metrics.pairwise import linear_kernel

In [33]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [34]:
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

In [35]:
cosine_sim.shape

(4795, 4795)

## Задание 5

Напиши функцию `get_recommendations`. На вход она принимает:
* `movies_dataset` - датасет фильмов
* `title` - название фильма, для которого мы будем искать похожие
* `cosine_sim` - матрица расстояний между описаниями
* `top_k` - топ-k cхожих фильмов

Возвращает top_k названий фильмов, описание которых похоже на выбранный фильм.\
Выведи топ-5 фильмов для `title='Saving Private Ryan'`

In [36]:
def get_recommendations(title, cosine_sim=cosine_sim):

    idx = indices[title]


    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)


    sim_scores = sim_scores[1:11]


    movie_indices = [i[0] for i in sim_scores]


    return df2['title'].iloc[movie_indices]

In [37]:
get_recommendations(title='Saving Private Ryan', cosine_sim=cosine_sim)

787         The Great Raid
586      The Monuments Men
290      The Expendables 2
4168             Abandoned
3516             The Train
1525        Apocalypse Now
2430             Evil Dead
3603     Lone Wolf McQuade
3586    As Above, So Below
755     Star Trek: Nemesis
Name: title, dtype: object

## Задание 6

Еще один подход к построению рекомендательной системы - подход на основе сходства между пользователями. Этот подход называется **Collaborative Filtering** (Коллаборативная фильтрация).
<center><img src="../misc/images/all.png" alt= “” width="600" height="700"></center>
Коллаборативная фильтрация - это тип рекомендательной системы, которая использует поведение и предпочтения похожих пользователей для рекомендации товаров или продуктов конкретному пользователю. Система собирает данные о прошлом поведении пользователей, такие как покупки, рейтинги и отзывы, и анализирует их для выявления закономерностей и сходства между пользователями. На основе этих закономерностей система рекомендует товары, которые понравились или были приобретены другими такими же пользователями в прошлом.

Для реализации Коллаборативной фильтрации нам потребуются оценки пользователей [ratings](../datasets/ratings.csv).

>userId - id пользователя \
movieId - id фильма \
rating - оценка фильма (от 0 до 5)\
timestamp - время оценки


Воспользуйся библиотекой [surprise](https://surpriselib.com/) для обучения модели оценки рейтинга фильма [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD). Выведи средние значения 'RMSE', 'MAE' на кросс-валидации с параметрами `cv=5`.

In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095444 sha256=1389fdfb18eb5d9beb866a79309b13091ee821e6b140f9bfb7ad95105661816a
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import Reader

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ratings=pd.read_csv('/content/ratings (1).csv')

In [ ]:
#создаем датасет, используя класс Reader, который определяет формат данных и их спецификации
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], Reader())

In [ ]:
#создаем обьект SVD(Singular Value Decomposision для рекомендательных систем)
algo=SVD()
#используем функцию cross_validate и выводим информацию о точности предсказаний для каждой метрики
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8959  0.8997  0.8989  0.9002  0.8959  0.8981  0.0019  
MAE (testset)     0.6907  0.6913  0.6933  0.6924  0.6879  0.6911  0.0018  
Fit time          1.02    1.04    1.03    1.02    1.01    1.02    0.01    
Test time         0.18    0.09    0.18    0.09    0.18    0.15    0.04    


{'test_rmse': array([0.89585101, 0.89969278, 0.89893857, 0.90023248, 0.89585419]),
 'test_mae': array([0.69065392, 0.69129327, 0.69334584, 0.69241641, 0.68793639]),
 'fit_time': (1.0219929218292236,
  1.035224199295044,
  1.0258991718292236,
  1.0196301937103271,
  1.0134942531585693),
 'test_time': (0.18103456497192383,
  0.09193229675292969,
  0.18232202529907227,
  0.09385061264038086,
  0.17592859268188477)}

Функция cross_validate принимает алгоритм, данные и списки метрик, используемых для оценки модели. В данном случае используются метрики RMSE (Mean Squared Error)и MAE (Mean Absolute Error) для оценки точночти предсказаний модели.
Функция также принимает параметр CV, который задает количество блоков для кросс-валидации и позволяет измерить стабильность модели.